In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import json
import os

In [4]:
# Paths
train_dir = r'/teamspace/studios/this_studio/mad/Dataset-MAD/train'
test_dir = r'/teamspace/studios/this_studio/mad/Dataset-MAD/test'
save_dir = r'/teamspace/studios/this_studio/model'

In [5]:
# Data Generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    validation_split=0.2
)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(299, 299),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)
validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(299, 299),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(299, 299),
    batch_size=32,
    class_mode='categorical'
)


Found 1377 images belonging to 3 classes.
Found 344 images belonging to 3 classes.
Found 431 images belonging to 3 classes.


In [6]:
# Model Definition
base_model = Xception(weights='imagenet', include_top=False, input_shape=(299, 299, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.4)(x)  # Reduced Dropout
predictions = Dense(train_generator.num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# Compile and Train Model
for layer in base_model.layers:
    layer.trainable = False
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=8,  # Adjusted Epochs
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

Epoch 1/8
43/43 ━━━━━━━━━━━━━━━━━━━━ 82s 2s/step - accuracy: 0.6998 - loss: 0.6526 - val_accuracy: 0.9375 - val_loss: 0.2028
Epoch 2/8
43/43 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - accuracy: 0.8438 - loss: 0.3158 - val_accuracy: 0.7917 - val_loss: 0.4203
Epoch 3/8
43/43 ━━━━━━━━━━━━━━━━━━━━ 80s 2s/step - accuracy: 0.9209 - loss: 0.2241 - val_accuracy: 0.9563 - val_loss: 0.1388
Epoch 4/8
43/43 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step - accuracy: 0.9375 - loss: 0.1869 - val_accuracy: 0.9167 - val_loss: 0.1571
Epoch 5/8
43/43 ━━━━━━━━━━━━━━━━━━━━ 80s 2s/step - accuracy: 0.9584 - loss: 0.1374 - val_accuracy: 0.9719 - val_loss: 0.1029
Epoch 6/8
43/43 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step - accuracy: 0.9688 - loss: 0.1296 - val_accuracy: 0.9167 - val_loss: 0.1319
Epoch 7/8


2024-10-27 10:35:22.282385: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


43/43 ━━━━━━━━━━━━━━━━━━━━ 81s 2s/step - accuracy: 0.9647 - loss: 0.1095 - val_accuracy: 0.9594 - val_loss: 0.0952
Epoch 8/8
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 1.0000 - loss: 0.0062 - val_accuracy: 0.9167 - val_loss: 0.0933


In [7]:
# Fine-Tune Last Few Layers
for layer in base_model.layers[-20:]:
    layer.trainable = True
model.compile(optimizer=Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=3,  # Adjusted Epochs
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

Epoch 1/3
43/43 ━━━━━━━━━━━━━━━━━━━━ 106s 2s/step - accuracy: 0.9238 - loss: 0.2031 - val_accuracy: 0.9625 - val_loss: 0.0722
Epoch 2/3
43/43 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step - accuracy: 0.9688 - loss: 0.2422 - val_accuracy: 1.0000 - val_loss: 0.0694
Epoch 3/3
43/43 ━━━━━━━━━━━━━━━━━━━━ 100s 2s/step - accuracy: 0.9455 - loss: 0.1356 - val_accuracy: 0.9750 - val_loss: 0.0592


In [8]:
# Save Model for TFLite Conversion
model_save_path = os.path.join(save_dir, 'pdd.h5')
model.save(model_save_path)

In [12]:
# Save the model in SavedModel format for TFLite conversion
model_save_path = os.path.join(save_dir, 'pdd')
model.export(model_save_path)  # Use export for SavedModel format

# Convert to TFLite with Quantization
converter = tf.lite.TFLiteConverter.from_saved_model(model_save_path)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()
tflite_model_save_path = os.path.join(save_dir, 'pdd.tflite')
with open(tflite_model_save_path, 'wb') as f:
    f.write(tflite_model)

print(f"TFLite model saved at {tflite_model_save_path}")




INFO:tensorflow:Assets written to: /teamspace/studios/this_studio/model/pdd/assets


INFO:tensorflow:Assets written to: /teamspace/studios/this_studio/model/pdd/assets


Saved artifact at '/teamspace/studios/this_studio/model/pdd'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 299, 299, 3), dtype=tf.float32, name='keras_tensor_136')
Output Type:
  TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)
Captures:
  140693537894848: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140689580758768: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140689580760704: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140689580756656: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140689580759824: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140689580766864: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140689580765808: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140689580768272: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140689580756832: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140689580767040: TensorSpec(shape=(), dtype=tf.resource,

W0000 00:00:1730026489.136839    5467 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1730026489.137004    5467 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2024-10-27 10:54:49.137702: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /teamspace/studios/this_studio/model/pdd
2024-10-27 10:54:49.145374: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2024-10-27 10:54:49.145395: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /teamspace/studios/this_studio/model/pdd
I0000 00:00:1730026489.207557    5467 mlir_graph_optimization_pass.cc:401] MLIR V1 optimization pass is not enabled
2024-10-27 10:54:49.220990: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2024-10-27 10:54:49.758958: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /teamspace/studios/this_studio/model/pdd
2024-10-27

In [13]:
# Save Class Indices as JSON for Kotlin Compatibility
class_indices_path = os.path.join(save_dir, 'class_indices.json')
with open(class_indices_path, 'w') as json_file:
    json.dump(train_generator.class_indices, json_file)

print(f"Model and class indices saved successfully in {save_dir}!")


Model and class indices saved successfully in /teamspace/studios/this_studio/model!
